In [1]:
import os
import sys
import jieba
import re
import numpy as np
from zhon.hanzi import punctuation #去除中文标点

# 第一步，构建语料库
选择除第一部小说之外的文本材料作为语料库来源，第一部作品作为测试文本

In [2]:
#文本替换
def reform(text):
    #首先去掉特殊符号以及数字字母
    text = re.sub( '[a-zA-Z0-9’!"#$%&\'()*+,-./:;<=>?@?★、…【】《》“”‘’[\\]^_`{|}~\s]+', "", text)
    #如果是以“。”结束的则将“。”删掉
    if (text.endswith("。") or text.endswith("？") or text.endswith("！")):
        text=text[:-1]
    #添加起始符BOS和终止符EOS   
    text1=text.replace("。", "EOSBOS").replace("!","EOSBOS").replace("，","EOSBOS").replace("？","EOSBOS")
    text2="BOS"+text1+"EOS"
    return text2

In [3]:
# 首先读取文本，去掉标点符号进行分割
word_lst = []
key_list = []
word_dict = {}

filepath = r'../..//NLP2020/dataset/inf.txt'
with open(filepath, 'r', encoding='utf-8') as f:
    index = f.read()
    index = index.split(",")
    for book in index[1:]:
        bookpath = r"../../NLP2020/dataset/"+book+".txt"
        with open(bookpath, 'r', encoding='utf-8') as f:
            print("正在读取："+book)
            text = f.read()
            text = reform(text)
            jieba.suggest_freq("BOS", True)
            jieba.suggest_freq("EOS", True)
            text = jieba.cut(text)
            for t in text:
                word_lst.append(t)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Reddish\AppData\Local\Temp\jieba.cache


正在读取：碧血剑


Loading model cost 0.691 seconds.
Prefix dict has been built successfully.


正在读取：飞狐外传
正在读取：连城诀
正在读取：鹿鼎记
正在读取：三十三剑客图
正在读取：射雕英雄传
正在读取：神雕侠侣
正在读取：书剑恩仇录
正在读取：天龙八部
正在读取：侠客行
正在读取：笑傲江湖
正在读取：雪山飞狐
正在读取：倚天屠龙记
正在读取：鸳鸯刀
正在读取：越女剑


In [4]:
# 统计一元词频
for item in word_lst:
    if item not in word_dict:
        word_dict[item] = 1
    else:
        word_dict[item] += 1

In [5]:
# 统计二元词频
word_dict_bi={}
def bilist(list_s):
    for i in range(0,len(list_s)-1):
        if (list_s[i+1],list_s[i]) in word_dict_bi:
            word_dict_bi[(list_s[i+1],list_s[i])] +=1
        else:
            word_dict_bi[(list_s[i+1],list_s[i])] = 1
    return  word_dict_bi

word_dict_bi = bilist(word_lst)

In [6]:
# 统计三元词频
word_dict_tri={}
def trilist(list_s):
    for i in range(0,len(list_s)-2):
        if (list_s[i+2],list_s[i+1],list_s[i]) in word_dict_tri:
            word_dict_tri[(list_s[i+2],list_s[i+1],list_s[i])] +=1
        else:
            word_dict_tri[(list_s[i+2],list_s[i+1],list_s[i])] =1
    return  word_dict_tri

word_dict_tri = trilist(word_lst)

# 第二步，读取测试文本

In [7]:
# 首先读取文本，去掉标点符号进行分割
word_lst_t = []
key_list_t = []
word_dict_t = {}

filepath = r'../..//NLP2020/dataset/inf.txt'
with open(filepath, 'r', encoding='utf-8') as f:
    index = f.read()
    index = index.split(",")
    book = index[0]
    bookpath = r"../../NLP2020/dataset/"+book+".txt"
    with open(bookpath, 'r', encoding='utf-8') as f:
        print("正在读取："+book)
        text = f.read()
        text = reform(text)
        jieba.suggest_freq("BOS", True)
        jieba.suggest_freq("EOS", True)
        text = jieba.cut(text)
        for t in text:
            word_lst_t.append(t)

正在读取：白马啸西风


# 第三步，计算交叉熵
$$H(X,M) = H(X)+KL(p||m)=-\sum_{x \to X}p(x)log(m(x))=E_p(-logm(x))=-\frac{1}{n}log m(x_1x_2...x_n)$$
$$m(x_1x_2...x_n)=m(x_1x_2)\prod_{i=3}^{n}m(x_i|x_{i-1}x_{i-2})=m(x_1x_2)m(x_3|x_1x_2)...m(x_n|x_{n-1}x_{n-2})$$

In [8]:
n = len(word_lst_t)
if (word_lst_t[1],word_lst_t[0]) in word_dict_bi:
    counts_bi= list(word_dict_bi.values())
    counts_bi= np.array(counts_bi).astype(float)
    sum_value =sum(word_dict_bi.values());
    jointp = np.log2(word_dict_bi[(word_lst_t[1],word_lst_t[0])] * 1.0 / sum_value)
else: 
    jointp = np.log2(0.000001);

for i in range(2,len(word_lst_t)):
    if ((word_lst_t[i],word_lst_t[i-1],word_lst_t[i-2]) not in word_dict_tri)or((word_lst_t[i-1],word_lst_t[i-2]) not in word_dict_bi):
        jointp += np.log2(0.000001)
    else:
        count_up = word_dict_tri[(word_lst_t[i],word_lst_t[i-1],word_lst_t[i-2])]
        count_down = word_dict_bi[(word_lst_t[i-1],word_lst_t[i-2])]
        jointp += np.log2((count_up * 1.0 / count_down))
cross_entropy = -jointp / n;
print("计算可得交叉熵为 "+str(cross_entropy)+" bit.")

计算可得交叉熵为 11.271312096091158 bit.


In [9]:
#文本替换
def reform(text):
    #首先去掉特殊符号以及数字字母
    text = re.sub( '[a-zA-Z0-9’!"#$%&\'()*+,-./:;<=>?@?★、…【】《》“”‘’[\\]^_`{|}~\s]+', "", text)
    #如果是以“。”结束的则将“。”删掉
    if (text.endswith("。") or text.endswith("？") or text.endswith("！")):
        text=text[:-1]
    #添加起始符BOS和终止符EOS   
    text1=text.replace("。", "EB").replace("!","EB").replace("，","EB").replace("？","EB")
    text2="B"+text1+"E"
    return text2

In [10]:
# 首先读取文本，去掉标点符号进行分割
word_lst_t = []
key_list_t = []
word_dict_t = {}

filepath = r'../..//NLP2020/dataset/inf.txt'
with open(filepath, 'r', encoding='utf-8') as f:
    index = f.read()
    index = index.split(",")
    book = index[0]
    bookpath = r"../../NLP2020/dataset/"+book+".txt"
    with open(bookpath, 'r', encoding='utf-8') as f:
        print("正在读取："+book)
        text = f.read()
        text = reform(text)
        for t in text:
            word_lst_t.append(t)

正在读取：白马啸西风


In [11]:
#平均词长
n2 = len(word_lst_t)
word_len = n2/n
word_len

1.4312860668000855

In [12]:
#字符交叉熵
cross_entropy/word_len

7.874954111228329